In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
#표를 불러오기 위한 라이브러리
from html_table_parser import parser_functions as parser
import collections
collections.Callable = collections.abc.Callable
#403 에러 방지
headers={"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}

In [2]:
#csv 파일 읽기

file_name = '반팔'+'_url_list.csv'
data = pd.read_csv(file_name, names=['name', 'url'])
urls=data['url'].values
print(urls.shape, urls)

(8911,) ['url' 'https://www.musinsa.com/app/goods/2368712'
 'https://www.musinsa.com/app/goods/2427585' ...
 'https://www.musinsa.com/app/goods/2000501'
 'https://www.musinsa.com/app/goods/1486747'
 'https://www.musinsa.com/app/goods/3504350']


In [ ]:
df=pd.DataFrame(data, columns=['name', 'url'])
url_list=df.values.tolist()

print(url_list)

In [11]:
url=url_list[1][1]

page = requests.get(url, headers = headers)
soup = bs(page.text, 'lxml')
table = soup.find_all(attrs={'class':'table_th_grey'})

#print(table)

p = parser.make2d(table[0])
df = pd.DataFrame(p[3:], columns=p[0])
print(df)

   cm  총장  어깨너비  가슴단면 소매길이
0  XS  58    44    50   19
1   S  67    48    52   22
2   M  70  50.5  55.5   24
3   L  72    52    58   25
4  XL  74  53.5  60.5   26


In [19]:
page = requests.get(url, headers = headers)
soup = bs(page.text, 'lxml')

i =0 

while i >= 0:
    try:
        review = soup.find_all(attrs={'class':'size_content'})[i].get_text()
        print("#: {}//{}".format(i, review))
        i+=1
    except:
        break


#: 0//
                                [회원추천] (여자 169cm/53kg) 기준
                                적당함 M Size 구매 

#: 1//
                                [회원추천] (남자 173cm/63kg) 기준
                                적당함 L Size 구매 

#: 2//
                                [회원추천] (여자 160cm/45kg) 기준
                                적당함 S Size 구매 

#: 3//
                                [회원추천] (여자 158cm/44kg) 기준
                                적당함 S Size 구매 

#: 4//
                                [회원추천] (여자 170cm/75kg) 기준
                                큼 L Size 구매 

#: 5//
                                [회원추천] (여자 165cm/55kg) 기준
                                적당함 S Size 구매 

#: 6//
                                [회원추천] (여자 170cm/63kg) 기준
                                큼 XL Size 구매 

#: 7//
                                [회원추천] (여자 158cm/50kg) 기준
                                적당함 S Size 구매 

#: 8//
                                [회원추천] (여자 155cm/50kg) 기준
                                적당